In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import scipy.io
import os
import requests
import tarfile


C:\Users\Valyr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # download the flowers dataset
url = "http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz"
filename = "flowers.tgz"
response = requests.get(url)
open(filename, "wb").write(response.content)
with tarfile.open(filename) as tar:
    tar.extractall()
os.remove(filename)

In [ ]:
# Load the image labels from the .mat file
url = "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat"
response = requests.get(url)

with open("imagelabels.mat", "wb") as f:
    f.write(response.content)

mat = scipy.io.loadmat("imagelabels.mat")
labels = mat['labels'].flatten()

In [ ]:
# Create a list of images and their corresponding labels
images = []
for i in range(len(labels)):
    image_path = os.path.join( 'jpg', f'image_{i + 1:05d}.jpg')
    if os.path.exists(image_path):
        images.append((image_path, labels[i-1]))

In [ ]:
# Create directories for each class
classes = set(labels)
for class_ in classes:
    class_dir = os.path.join('flowers', 'train', str(class_))
    os.makedirs(class_dir, exist_ok=True)


In [ ]:
# Link the images to their respective class directories
for image_path, label in images:
    target_path = os.path.join('flowers', 'train', str(label), os.path.basename(image_path))
    i = 0
    while os.path.exists(target_path):
        filename, ext = os.path.splitext(os.path.basename(target_path))
        target_path = os.path.join('flowers', 'train', str(label), f'{filename}{ext}')
        i += 1
    os.link(image_path, target_path)

In [ ]:
# Load the Flowers dataset with the specified transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder(root=os.path.join('flowers', 'train'), transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [5]:
# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Load the pre-trained ResNet 50 model
model = torchvision.models.resnet50(pretrained=True)

# Freeze all layers except the last one
# This is done to prevent the model's pre-trained parameters from being changed during training.
# The idea is to keep the feature extraction part intact and only update the fully connected layer
for param in model.parameters():
    param.requiresGrad = False
model.fc.requiresGrad = True

# Set up the loss function and optimizer
# CrossEntropyLoss is used for multi-class classification problems
# SGD is used as the optimizer with a learning rate of 0.001 and momentum of 0.9
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train the model for 10 epochs
for epoch in range(10): 
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the gradients before each iteration
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()
    print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss / len(dataloader)))

print('Finished Training')